In [1]:
# load the model first, run the evaluator on the model.

import pandas as pd
import numpy as np
import torch
import utils.print as print_f
import os

from utils.engine import evaluate, get_iou_types
from utils.plot import plot_losses, plot_performance

from models.utils import get_model_size_in_MB
from models.build import create_model_from_setup
from models.setup import ModelSetup
from models.train import TrainingInfo
from utils.save import check_best, end_train, get_data_from_metric_logger
from data.load import get_datasets, get_dataloaders
from IPython.display import clear_output
from utils.eval import get_ap_ar, get_ap_ar_for_train_val
from utils.train import get_optimiser, get_lr_scheduler, print_params_setup, get_coco_eval_params, get_dynamic_loss
from utils.init import reproducibility, clean_memory_get_device
from data.paths import MIMIC_EYE_PATH
# from datetime import datetime
# from models.dynamic_loss import DynamicWeightedLoss
from utils.engine import train_one_epoch, evaluate
from torch import optim
from data.paths import MIMIC_EYE_PATH
from models.load import get_trained_model
# from data.load import seed_worker, get_dataloader_g
from utils.plot import DISEASE_CMAP, get_legend_elements
legend_elements = get_legend_elements(DISEASE_CMAP["solid"])

## Suppress the assignement warning from pandas.r
pd.options.mode.chained_assignment = None  # default='warn'

## Supress user warning
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
from enum import Enum
class TrainedModels(Enum):
    mobilenet_with_fix = "val_lesion-detection_froc_0_7916_test_lesion-detection_froc_0_8150_epoch15_04-01-2023 03-02-43_lesion_dsetection_with_fix_mobilenet"
    mobilenet_baseline = "val_lesion-detection_froc_0_7764_test_lesion-detection_froc_0_8036_epoch8_04-01-2023 07-46-01_lesion_dsetection_baseline_mobilenet"
    resnet18_baseline = "val_lesion-detection_froc_0_7893_test_lesion-detection_froc_0_8075_epoch5_04-01-2023 13-53-42_lesion_dsetection_baseline_resnet18"
    resnet18_with_fix = "val_lesion-detection_froc_0_7982_test_lesion-detection_froc_0_8257_epoch5_04-01-2023 11-24-08_lesion_dsetection_with_fix_resnet18"

In [3]:
# expectation in froc

# 1. test if we have the same value for repeating the froc calculation in the test.ipynb
# 2. test why the training (end train) and here has different values shown.

In [4]:
device = clean_memory_get_device()
reproducibility()

This notebook will running on device: [CUDA]


In [5]:
get_epoch_from_model = lambda select_model: int(([substr for substr in select_model.value.split("_") if "epoch" in substr][0]).replace("epoch", ""))
get_model_name = lambda select_model: str(select_model).split(".")[-1]

In [6]:
select_model = TrainedModels.resnet18_with_fix

In [24]:
model, train_info, optimizer, dynamic_loss_weight = get_trained_model(
        select_model,
        device,
    )

model.to(device)

# setting up the score threshold to 0 for froc.
model.task_performers['lesion-detection'].roi_heads.score_thresh = 0
model.eval()

# cover the long model.
print()

Using pretrained backbone. resnet18
Using pretrained backbone. resnet18
Using SGD as optimizer with lr=0.01



In [25]:
setup = train_info.model_setup
iou_types = get_iou_types(model, setup)

In [26]:
from data.strs import SourceStrs, TaskStrs

dataset_params_dict = {
        "MIMIC_EYE_PATH": MIMIC_EYE_PATH,
        "labels_cols": setup.lesion_label_cols,
        "with_xrays_input": SourceStrs.XRAYS in setup.sources,
        "with_clincal_input": SourceStrs.CLINICAL in setup.sources,
        "with_fixations_input": SourceStrs.FIXATIONS in setup.sources,
        "fixations_mode_input": setup.fiaxtions_mode_input,
        "with_bboxes_label": TaskStrs.LESION_DETECTION in setup.tasks,
        "with_fixations_label": TaskStrs.FIXATION_GENERATION in setup.tasks,
        "fixations_mode_label": setup.fiaxtions_mode_label,
        "with_chexpert_label": TaskStrs.CHEXPERT_CLASSIFICATION in setup.tasks,
        "with_negbio_label": TaskStrs.NEGBIO_CLASSIFICATION in setup.tasks,
        "clinical_numerical_cols": setup.clinical_num,
        "clinical_categorical_cols": setup.clinical_cat,
        "image_size": setup.image_size,
        "image_mean": setup.image_mean,
        "image_std": setup.image_std,
    }

In [27]:
detect_eval_dataset, train_dataset, val_dataset, test_dataset = get_datasets(
    dataset_params_dict=dataset_params_dict,
)

train_dataloader, val_dataloader, test_dataloader = get_dataloaders(
    train_dataset, val_dataset, test_dataset, batch_size=setup.batch_size,
)

train_coco = None
train_coco, val_coco, test_coco, eval_dict = get_coco_eval_params(
    source_name=SourceStrs.XRAYS,
    task_name=TaskStrs.LESION_DETECTION,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    test_dataloader=test_dataloader,
    detect_eval_dataset=detect_eval_dataset,
    iou_thrs=np.array([0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]),
    use_iobb=setup.use_iobb,
    maxDets=[1, 5, 10, 30],
)


In [28]:
model.task_performers["lesion-detection"].roi_heads.score_thresh = 0
test_evaluator, _ = evaluate(
    setup=setup,
    model=model,
    data_loader=test_dataloader,
    device=device,
    params_dict=eval_dict,
    coco=test_coco,
    iou_types=iou_types,
    return_dt_gt=True,
)
model.task_performers["lesion-detection"].roi_heads.score_thresh = 0.05

Evaluation:  [ 0/57]  eta: 0:02:22  loss: 0.2886 (0.2886)  lesion-detection_performer-object_detection_loss_classifier: 0.1451 (0.1451)  lesion-detection_performer-object_detection_loss_box_reg: 0.1070 (0.1070)  lesion-detection_performer-object_detection_loss_objectness: 0.0321 (0.0321)  lesion-detection_performer-object_detection_loss_rpn_box_reg: 0.0044 (0.0044)  model_time: 1680353408.0000 (1680353359.5199)  evaluator_time: 0.0000 (0.0000)  time: 2.4994  data: 2.0872  max mem: 646
Evaluation:  [56/57]  eta: 0:00:02  loss: 0.2060 (0.2031)  lesion-detection_performer-object_detection_loss_classifier: 0.1091 (0.1069)  lesion-detection_performer-object_detection_loss_box_reg: 0.0798 (0.0739)  lesion-detection_performer-object_detection_loss_objectness: 0.0180 (0.0190)  lesion-detection_performer-object_detection_loss_rpn_box_reg: 0.0034 (0.0033)  model_time: 1680353408.0000 (1680353419.1402)  evaluator_time: 0.0000 (0.0000)  time: 2.1465  data: 1.9474  max mem: 646
Evaluation: Total ti

In [29]:
from coco_froc_analysis.froc.froc_curve import get_froc_curve, get_interpolate_froc

In [30]:
stats, lls_accuracy, nlls_per_image = get_froc_curve(
    dataset=test_dataset,
    dts=test_evaluator['lesion-detection'].all_dts,
    all_gts=test_evaluator['lesion-detection'].all_gts,
    plot_title= None,
    use_iou=True,
    n_sample_points=200,
    froc_save_folder="./froc_figures"
)

  0%|          | 0/200 [00:00<?, ?it/s]

In [31]:
froc_v = get_interpolate_froc(
    stats=stats,
    lls_accuracy=lls_accuracy,
    nlls_per_image=nlls_per_image,
    cat_id=None,
    fps_per_img=[0.5, 1, 2, 4],
    weight=True,
)

In [32]:
froc_v

array([0.5701297 , 0.79008697, 0.95343098, 0.99134157])

In [33]:
np.mean(froc_v)

0.8262473052117743

In [17]:
# 0.8066839859865023

# why this one is different?
# 0.8270512150394913 -> with fix attempt 1
# 0.8234257443138374 -> with fix attempt 2 
# 0.8234257443138374 -> with fix attempt 3
# 0.8234257443138374 -> 
# 0.8234257443138374 
# 0.8234257443138374

#baseline 
# 0.8066839859865023
# 0.8066839859865023
# 0.8089129224630301
# 0.8065924636538497
#reload model and re-run


# 0.8234257443138374
#no-restart kernal.
# 0.8246889953396321



# what if I reconstruct eval_params?
# what if I run another model first? 

# 0.8075 -> expectation. [baseline.]
# 0.8275 -> with fixation. 